# Module 7: Autoencoder 與 VAE

## 學習目標
- 理解 Autoencoder 的編碼器-解碼器架構
- 掌握潛在空間 (Latent Space) 的概念
- 學習 VAE 的數學原理與實現
- 使用 VAE 生成 MNIST 手寫數字

## 為什麼要學 Autoencoder？

Autoencoder 是**生成模型**的基礎，理解它能幫助你：
1. 資料壓縮與降維
2. 特徵學習
3. 異常檢測
4. 圖像生成（VAE、Diffusion 的基礎）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

# 設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 可視化設定
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)

---
## Part 1: Autoencoder 基礎

### 直觀理解

Autoencoder 就像是一個「壓縮-解壓」系統：

```
輸入 x ──► [編碼器] ──► 潛在表示 z ──► [解碼器] ──► 重建 x'
(784)       壓縮         (32)          解壓        (784)

目標：讓 x' 盡可能接近 x
```

**為什麼有用？**
- 潛在空間 z 捕捉了資料的**本質特徵**
- 強迫網路學習重要的表示（降維）

In [ ]:
class SimpleAutoencoder(nn.Module):
    """
    最簡單的全連接 Autoencoder
    
    結構：
    - 編碼器：784 -> 256 -> 64 -> latent_dim
    - 解碼器：latent_dim -> 64 -> 256 -> 784
    """
    
    def __init__(self, latent_dim: int = 32):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        # 編碼器：將輸入壓縮到潛在空間
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim),
        )
        
        # 解碼器：從潛在空間重建輸入
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid(),  # 輸出在 [0, 1] 範圍
        )
        
    def encode(self, x: torch.Tensor) -> torch.Tensor:
        """編碼：x -> z"""
        x = x.view(-1, 784)  # 展平
        return self.encoder(x)
    
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """解碼：z -> x'"""
        return self.decoder(z)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """完整前向傳播"""
        z = self.encode(x)
        x_recon = self.decode(z)
        return x_recon, z

# 測試
model = SimpleAutoencoder(latent_dim=32)
x = torch.randn(4, 1, 28, 28)  # 模擬 MNIST 輸入
x_recon, z = model(x)

print("Simple Autoencoder 測試：")
print(f"輸入形狀: {x.shape}")
print(f"潛在表示形狀: {z.shape}")
print(f"重建輸出形狀: {x_recon.shape}")
print(f"\n壓縮比: {784 / 32:.1f}x (784 -> 32)")

### 載入 MNIST 資料

In [ ]:
# 資料載入
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"訓練集大小: {len(train_dataset)}")
print(f"測試集大小: {len(test_dataset)}")

# 顯示一些樣本
fig, axes = plt.subplots(2, 10, figsize=(12, 3))
for i in range(10):
    idx = np.random.randint(len(train_dataset))
    img, label = train_dataset[idx]
    axes[0, i].imshow(img.squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[0, i].set_title(str(label))
    
    idx = np.random.randint(len(train_dataset))
    img, label = train_dataset[idx]
    axes[1, i].imshow(img.squeeze(), cmap='gray')
    axes[1, i].axis('off')
    axes[1, i].set_title(str(label))

plt.suptitle('MNIST 樣本')
plt.tight_layout()
plt.show()

### 訓練 Autoencoder

In [ ]:
def train_autoencoder(model, train_loader, epochs=10, lr=1e-3):
    """
    訓練 Autoencoder
    
    損失函數：重建損失（MSE 或 BCE）
    目標：最小化輸入與重建之間的差異
    """
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # 使用 BCE 損失（因為像素值在 [0,1]）
    criterion = nn.BCELoss(reduction='sum')
    
    history = {'loss': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device)
            
            # 前向傳播
            recon, _ = model(data)
            
            # 計算重建損失
            loss = criterion(recon, data.view(-1, 784))
            
            # 反向傳播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader.dataset)
        history['loss'].append(avg_loss)
        
        if (epoch + 1) % 2 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return history

# 訓練
ae_model = SimpleAutoencoder(latent_dim=32).to(device)
history = train_autoencoder(ae_model, train_loader, epochs=10)

In [ ]:
def visualize_reconstruction(model, test_loader, n_samples=10):
    """
    可視化重建結果
    """
    model.eval()
    
    # 取得一批測試資料
    data, _ = next(iter(test_loader))
    data = data[:n_samples].to(device)
    
    with torch.no_grad():
        recon, _ = model(data)
    
    # 繪製
    fig, axes = plt.subplots(2, n_samples, figsize=(15, 3))
    
    for i in range(n_samples):
        # 原始圖像
        axes[0, i].imshow(data[i].cpu().squeeze(), cmap='gray')
        axes[0, i].axis('off')
        if i == 0:
            axes[0, i].set_title('原始', fontsize=12)
        
        # 重建圖像
        axes[1, i].imshow(recon[i].cpu().view(28, 28), cmap='gray')
        axes[1, i].axis('off')
        if i == 0:
            axes[1, i].set_title('重建', fontsize=12)
    
    plt.suptitle('Autoencoder 重建效果')
    plt.tight_layout()
    plt.show()

visualize_reconstruction(ae_model, test_loader)

### 潛在空間可視化

In [ ]:
def visualize_latent_space(model, test_loader, n_samples=3000):
    """
    將潛在空間用 t-SNE 或 PCA 降維後可視化
    """
    from sklearn.decomposition import PCA
    
    model.eval()
    
    latents = []
    labels_list = []
    
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.to(device)
            _, z = model(data)
            latents.append(z.cpu())
            labels_list.append(labels)
            
            if len(latents) * data.shape[0] >= n_samples:
                break
    
    latents = torch.cat(latents, dim=0)[:n_samples].numpy()
    labels_list = torch.cat(labels_list, dim=0)[:n_samples].numpy()
    
    # PCA 降到 2D
    pca = PCA(n_components=2)
    latents_2d = pca.fit_transform(latents)
    
    # 繪製
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(latents_2d[:, 0], latents_2d[:, 1], 
                         c=labels_list, cmap='tab10', alpha=0.6, s=10)
    plt.colorbar(scatter, label='數字類別')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('Autoencoder 潛在空間 (PCA 投影)')
    plt.show()
    
    print(f"PCA 解釋變異比: {pca.explained_variance_ratio_.sum()*100:.1f}%")

visualize_latent_space(ae_model, test_loader)

---
## Part 2: 卷積 Autoencoder

對於圖像，使用卷積層效果更好：

In [ ]:
class ConvAutoencoder(nn.Module):
    """
    卷積 Autoencoder
    
    使用卷積進行編碼，轉置卷積進行解碼
    """
    
    def __init__(self, latent_dim: int = 32):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        # 編碼器
        # 輸入: [B, 1, 28, 28]
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),  # [B, 32, 14, 14]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # [B, 64, 7, 7]
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        # 全連接到潛在空間
        self.encoder_fc = nn.Linear(64 * 7 * 7, latent_dim)
        
        # 從潛在空間解碼
        self.decoder_fc = nn.Linear(latent_dim, 64 * 7 * 7)
        
        # 解碼器
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 32, 14, 14]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 1, 28, 28]
            nn.Sigmoid(),
        )
        
    def encode(self, x: torch.Tensor) -> torch.Tensor:
        h = self.encoder_conv(x)
        h = h.view(h.size(0), -1)  # 展平
        z = self.encoder_fc(h)
        return z
    
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        h = self.decoder_fc(z)
        h = h.view(h.size(0), 64, 7, 7)  # 重塑
        x_recon = self.decoder_conv(h)
        return x_recon
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        z = self.encode(x)
        x_recon = self.decode(z)
        return x_recon, z

# 測試
conv_ae = ConvAutoencoder(latent_dim=32)
x = torch.randn(4, 1, 28, 28)
x_recon, z = conv_ae(x)

print("Conv Autoencoder 測試：")
print(f"輸入形狀: {x.shape}")
print(f"潛在表示形狀: {z.shape}")
print(f"重建輸出形狀: {x_recon.shape}")

In [ ]:
# 訓練卷積 Autoencoder
def train_conv_autoencoder(model, train_loader, epochs=10, lr=1e-3):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()  # 使用 MSE
    
    history = {'loss': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for data, _ in train_loader:
            data = data.to(device)
            
            recon, _ = model(data)
            loss = criterion(recon, data)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * data.size(0)
        
        avg_loss = total_loss / len(train_loader.dataset)
        history['loss'].append(avg_loss)
        
        if (epoch + 1) % 2 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")
    
    return history

conv_ae_model = ConvAutoencoder(latent_dim=32).to(device)
history = train_conv_autoencoder(conv_ae_model, train_loader, epochs=10)

In [ ]:
# 比較重建效果
def compare_reconstruction(fc_model, conv_model, test_loader, n_samples=10):
    fc_model.eval()
    conv_model.eval()
    
    data, _ = next(iter(test_loader))
    data = data[:n_samples].to(device)
    
    with torch.no_grad():
        recon_fc, _ = fc_model(data)
        recon_conv, _ = conv_model(data)
    
    fig, axes = plt.subplots(3, n_samples, figsize=(15, 4.5))
    
    for i in range(n_samples):
        # 原始
        axes[0, i].imshow(data[i].cpu().squeeze(), cmap='gray')
        axes[0, i].axis('off')
        if i == 0:
            axes[0, i].set_ylabel('原始', fontsize=12)
        
        # FC 重建
        axes[1, i].imshow(recon_fc[i].cpu().view(28, 28), cmap='gray')
        axes[1, i].axis('off')
        if i == 0:
            axes[1, i].set_ylabel('FC AE', fontsize=12)
        
        # Conv 重建
        axes[2, i].imshow(recon_conv[i].cpu().squeeze(), cmap='gray')
        axes[2, i].axis('off')
        if i == 0:
            axes[2, i].set_ylabel('Conv AE', fontsize=12)
    
    plt.suptitle('FC vs Conv Autoencoder 重建比較')
    plt.tight_layout()
    plt.show()

compare_reconstruction(ae_model, conv_ae_model, test_loader)

---
## Part 3: Variational Autoencoder (VAE)

### VAE 的直觀理解

普通 Autoencoder 的問題：潛在空間不連續、不規則

```
普通 AE 的潛在空間：
    ┌───────────────────┐
    │  •••   ∘∘∘       │   不同數字聚在一起
    │    ×××    ○○     │   但空間有「空洞」
    │  △△△         ◇◇ │   從空洞採樣會得到奇怪結果
    └───────────────────┘

VAE 的潛在空間：
    ┌───────────────────┐
    │   ∘•×△○◇ 平滑過渡  │   強制潛在空間接近標準正態分佈
    │  • ∘ × △ ○ ◇     │   可以從任意位置採樣
    │ • ∘ × △ ○ ◇      │   得到有意義的輸出
    └───────────────────┘
```

### VAE 的關鍵改變

1. **編碼器輸出分佈參數**而非點：
   - 輸出 $\mu$ 和 $\sigma$（均值和標準差）
   - $z \sim \mathcal{N}(\mu, \sigma^2)$

2. **增加 KL 散度損失**：
   - 強制潛在分佈接近標準正態分佈 $\mathcal{N}(0, 1)$

### VAE 數學

**損失函數** = 重建損失 + KL 散度

$$\mathcal{L} = \mathbb{E}_{z \sim q(z|x)}[\log p(x|z)] - \text{KL}(q(z|x) || p(z))$$

其中：
- 第一項：重建損失（希望從 z 能重建出 x）
- 第二項：KL 散度（希望 q(z|x) 接近先驗 p(z) = N(0,1)）

**KL 散度的解析解**（當 q(z|x) 是高斯分佈時）：

$$\text{KL} = -\frac{1}{2} \sum_{j=1}^{J} (1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2)$$

In [ ]:
class VAE(nn.Module):
    """
    Variational Autoencoder
    
    關鍵改變：
    1. 編碼器輸出 mu 和 log_var
    2. 使用重參數化技巧進行採樣
    """
    
    def __init__(self, latent_dim: int = 32):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        # 編碼器
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        # 輸出 mu 和 log_var
        self.fc_mu = nn.Linear(64 * 7 * 7, latent_dim)
        self.fc_logvar = nn.Linear(64 * 7 * 7, latent_dim)
        
        # 解碼器
        self.decoder_fc = nn.Linear(latent_dim, 64 * 7 * 7)
        
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )
    
    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        編碼：返回分佈參數 (mu, log_var)
        """
        h = self.encoder_conv(x)
        h = h.view(h.size(0), -1)
        
        mu = self.fc_mu(h)
        log_var = self.fc_logvar(h)
        
        return mu, log_var
    
    def reparameterize(self, mu: torch.Tensor, log_var: torch.Tensor) -> torch.Tensor:
        """
        重參數化技巧 (Reparameterization Trick)
        
        z = mu + sigma * epsilon, 其中 epsilon ~ N(0, 1)
        
        這讓採樣過程可微分！
        """
        std = torch.exp(0.5 * log_var)  # sigma = exp(0.5 * log(sigma^2))
        epsilon = torch.randn_like(std)  # 從標準正態採樣
        z = mu + std * epsilon
        return z
    
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        h = self.decoder_fc(z)
        h = h.view(h.size(0), 64, 7, 7)
        return self.decoder_conv(h)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var

# 測試
vae = VAE(latent_dim=32)
x = torch.randn(4, 1, 28, 28)
x_recon, mu, log_var = vae(x)

print("VAE 測試：")
print(f"輸入形狀: {x.shape}")
print(f"mu 形狀: {mu.shape}")
print(f"log_var 形狀: {log_var.shape}")
print(f"重建輸出形狀: {x_recon.shape}")

### VAE 損失函數

In [ ]:
def vae_loss(recon_x, x, mu, log_var, kl_weight=1.0):
    """
    VAE 損失 = 重建損失 + KL 散度
    
    Args:
        recon_x: 重建圖像 [B, 1, 28, 28]
        x: 原始圖像 [B, 1, 28, 28]
        mu: 編碼器輸出的均值 [B, latent_dim]
        log_var: 編碼器輸出的對數方差 [B, latent_dim]
        kl_weight: KL 項的權重（可用於 beta-VAE）
    """
    
    # 重建損失 (Binary Cross Entropy)
    recon_loss = F.binary_cross_entropy(recon_x, x, reduction='sum')
    
    # KL 散度: -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    
    total_loss = recon_loss + kl_weight * kl_loss
    
    return total_loss, recon_loss, kl_loss

# 測試損失計算
x = torch.rand(4, 1, 28, 28)  # 注意：BCE 需要輸入在 [0,1]
vae = VAE(latent_dim=32)
x_recon, mu, log_var = vae(x)

total, recon, kl = vae_loss(x_recon, x, mu, log_var)
print(f"總損失: {total.item():.2f}")
print(f"重建損失: {recon.item():.2f}")
print(f"KL 散度: {kl.item():.2f}")

### 訓練 VAE

In [ ]:
def train_vae(model, train_loader, epochs=15, lr=1e-3, kl_weight=1.0):
    """
    訓練 VAE
    """
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    history = {'total_loss': [], 'recon_loss': [], 'kl_loss': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_recon = 0
        total_kl = 0
        
        for data, _ in train_loader:
            data = data.to(device)
            
            # 前向傳播
            recon, mu, log_var = model(data)
            
            # 計算損失
            loss, recon_loss, kl_loss = vae_loss(recon, data, mu, log_var, kl_weight)
            
            # 反向傳播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_recon += recon_loss.item()
            total_kl += kl_loss.item()
        
        n = len(train_loader.dataset)
        history['total_loss'].append(total_loss / n)
        history['recon_loss'].append(total_recon / n)
        history['kl_loss'].append(total_kl / n)
        
        if (epoch + 1) % 3 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Total: {history['total_loss'][-1]:.2f}, "
                  f"Recon: {history['recon_loss'][-1]:.2f}, "
                  f"KL: {history['kl_loss'][-1]:.2f}")
    
    return history

# 訓練
vae_model = VAE(latent_dim=32).to(device)
vae_history = train_vae(vae_model, train_loader, epochs=15, kl_weight=1.0)

In [ ]:
# 繪製訓練曲線
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(vae_history['total_loss'])
axes[0].set_title('總損失')
axes[0].set_xlabel('Epoch')

axes[1].plot(vae_history['recon_loss'])
axes[1].set_title('重建損失')
axes[1].set_xlabel('Epoch')

axes[2].plot(vae_history['kl_loss'])
axes[2].set_title('KL 散度')
axes[2].set_xlabel('Epoch')

plt.tight_layout()
plt.show()

### VAE 生成新樣本

In [ ]:
def generate_samples(model, n_samples=20):
    """
    從 VAE 生成新樣本
    
    步驟：從標準正態分佈採樣 z，然後解碼
    """
    
    model.eval()
    
    with torch.no_grad():
        # 從標準正態分佈採樣
        z = torch.randn(n_samples, model.latent_dim).to(device)
        
        # 解碼生成圖像
        samples = model.decode(z)
    
    # 可視化
    n_cols = 10
    n_rows = (n_samples + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 1.5 * n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for i in range(n_samples):
        axes[i].imshow(samples[i].cpu().squeeze(), cmap='gray')
        axes[i].axis('off')
    
    # 隱藏多餘的子圖
    for i in range(n_samples, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('VAE 生成的新樣本（從 N(0,1) 採樣 z）')
    plt.tight_layout()
    plt.show()

generate_samples(vae_model, n_samples=20)

### 潛在空間探索

In [ ]:
def visualize_vae_latent_space(model, test_loader, n_samples=5000):
    """
    可視化 VAE 的潛在空間
    
    與普通 AE 比較，VAE 的潛在空間應該更規整
    """
    from sklearn.decomposition import PCA
    
    model.eval()
    
    mus = []
    labels_list = []
    
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.to(device)
            mu, _ = model.encode(data)
            mus.append(mu.cpu())
            labels_list.append(labels)
            
            if len(mus) * data.shape[0] >= n_samples:
                break
    
    mus = torch.cat(mus, dim=0)[:n_samples].numpy()
    labels_list = torch.cat(labels_list, dim=0)[:n_samples].numpy()
    
    # PCA 降維
    pca = PCA(n_components=2)
    mus_2d = pca.fit_transform(mus)
    
    # 繪製
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(mus_2d[:, 0], mus_2d[:, 1], 
                         c=labels_list, cmap='tab10', alpha=0.6, s=10)
    plt.colorbar(scatter, label='數字類別')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('VAE 潛在空間 (mu 的 PCA 投影)')
    
    # 添加參考圓（標準正態分佈的等高線）
    circle = plt.Circle((0, 0), 2, fill=False, linestyle='--', color='gray', alpha=0.5)
    plt.gca().add_patch(circle)
    
    plt.show()

visualize_vae_latent_space(vae_model, test_loader)

In [ ]:
def latent_space_interpolation(model, test_loader):
    """
    潛在空間插值：展示兩個數字之間的平滑過渡
    """
    
    model.eval()
    
    # 取得兩個不同的樣本
    data, labels = next(iter(test_loader))
    
    # 找到兩個不同數字
    idx1, idx2 = 0, 1
    while labels[idx1] == labels[idx2]:
        idx2 += 1
    
    x1 = data[idx1:idx1+1].to(device)
    x2 = data[idx2:idx2+1].to(device)
    
    with torch.no_grad():
        mu1, _ = model.encode(x1)
        mu2, _ = model.encode(x2)
        
        # 在潛在空間中進行線性插值
        n_steps = 10
        interpolations = []
        
        for alpha in np.linspace(0, 1, n_steps):
            z = (1 - alpha) * mu1 + alpha * mu2
            x_interp = model.decode(z)
            interpolations.append(x_interp)
        
        interpolations = torch.cat(interpolations, dim=0)
    
    # 可視化
    fig, axes = plt.subplots(1, n_steps + 2, figsize=(14, 1.5))
    
    # 原始圖像 1
    axes[0].imshow(x1.cpu().squeeze(), cmap='gray')
    axes[0].axis('off')
    axes[0].set_title(f'{labels[idx1].item()}')
    
    # 插值結果
    for i in range(n_steps):
        axes[i+1].imshow(interpolations[i].cpu().squeeze(), cmap='gray')
        axes[i+1].axis('off')
    
    # 原始圖像 2
    axes[-1].imshow(x2.cpu().squeeze(), cmap='gray')
    axes[-1].axis('off')
    axes[-1].set_title(f'{labels[idx2].item()}')
    
    plt.suptitle('VAE 潛在空間插值')
    plt.tight_layout()
    plt.show()

latent_space_interpolation(vae_model, test_loader)

### 2D 潛在空間的 VAE（更好的可視化）

In [ ]:
# 訓練一個 2D 潛在空間的 VAE
vae_2d = VAE(latent_dim=2).to(device)
vae_2d_history = train_vae(vae_2d, train_loader, epochs=20, kl_weight=1.0)

In [ ]:
def visualize_2d_latent_manifold(model, n_points=20, range_val=3):
    """
    在 2D 潛在空間中網格採樣，生成圖像流形
    """
    
    model.eval()
    
    # 建立網格
    z1 = np.linspace(-range_val, range_val, n_points)
    z2 = np.linspace(-range_val, range_val, n_points)
    
    # 生成圖像
    canvas = np.zeros((28 * n_points, 28 * n_points))
    
    with torch.no_grad():
        for i, z2_val in enumerate(z2[::-1]):  # 上到下
            for j, z1_val in enumerate(z1):     # 左到右
                z = torch.tensor([[z1_val, z2_val]], dtype=torch.float32).to(device)
                x = model.decode(z)
                canvas[i*28:(i+1)*28, j*28:(j+1)*28] = x.cpu().squeeze().numpy()
    
    plt.figure(figsize=(12, 12))
    plt.imshow(canvas, cmap='gray')
    plt.xlabel('z1')
    plt.ylabel('z2')
    plt.title('VAE 2D 潛在空間流形')
    
    # 添加座標標籤
    plt.xticks(np.linspace(0, 28*n_points, 5), 
               [f'{v:.1f}' for v in np.linspace(-range_val, range_val, 5)])
    plt.yticks(np.linspace(0, 28*n_points, 5), 
               [f'{v:.1f}' for v in np.linspace(range_val, -range_val, 5)])
    
    plt.show()

visualize_2d_latent_manifold(vae_2d, n_points=15, range_val=3)

In [ ]:
def plot_2d_latent_distribution(model, test_loader):
    """
    繪製 2D 潛在空間中各類別的分佈
    """
    
    model.eval()
    
    mus = []
    labels_list = []
    
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.to(device)
            mu, _ = model.encode(data)
            mus.append(mu.cpu())
            labels_list.append(labels)
    
    mus = torch.cat(mus, dim=0).numpy()
    labels_list = torch.cat(labels_list, dim=0).numpy()
    
    # 繪製
    plt.figure(figsize=(10, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    
    for digit in range(10):
        mask = labels_list == digit
        plt.scatter(mus[mask, 0], mus[mask, 1], 
                   c=[colors[digit]], alpha=0.5, s=10, label=str(digit))
    
    plt.xlabel('z1')
    plt.ylabel('z2')
    plt.title('VAE 2D 潛在空間分佈')
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    
    # 添加標準正態參考
    circle = plt.Circle((0, 0), 2, fill=False, linestyle='--', color='gray', alpha=0.5)
    plt.gca().add_patch(circle)
    
    plt.axis('equal')
    plt.show()

plot_2d_latent_distribution(vae_2d, test_loader)

---
## Part 4: Beta-VAE 與 Disentangled 表示

Beta-VAE 透過增加 KL 項的權重來學習「解糾纏」的表示：

In [ ]:
# 比較不同 beta 值的效果
def train_and_compare_betas(betas=[0.1, 1.0, 4.0], epochs=10):
    """
    訓練不同 beta 值的 VAE 並比較結果
    
    beta < 1: 更好的重建，但潛在空間可能不規整
    beta = 1: 標準 VAE
    beta > 1: 更規整的潛在空間，但重建可能模糊
    """
    
    models = {}
    histories = {}
    
    for beta in betas:
        print(f"\n訓練 beta={beta} 的 VAE...")
        model = VAE(latent_dim=32).to(device)
        history = train_vae(model, train_loader, epochs=epochs, kl_weight=beta)
        models[beta] = model
        histories[beta] = history
    
    return models, histories

# 訓練（可能需要幾分鐘）
beta_models, beta_histories = train_and_compare_betas([0.1, 1.0, 4.0], epochs=10)

In [ ]:
def compare_beta_reconstructions(models, test_loader):
    """
    比較不同 beta VAE 的重建效果
    """
    
    data, _ = next(iter(test_loader))
    data = data[:8].to(device)
    
    betas = list(models.keys())
    n_samples = data.shape[0]
    
    fig, axes = plt.subplots(len(betas) + 1, n_samples, figsize=(12, 4))
    
    # 原始圖像
    for i in range(n_samples):
        axes[0, i].imshow(data[i].cpu().squeeze(), cmap='gray')
        axes[0, i].axis('off')
    axes[0, 0].set_ylabel('原始', fontsize=10)
    
    # 各 beta 的重建
    for row, beta in enumerate(betas, 1):
        model = models[beta]
        model.eval()
        
        with torch.no_grad():
            recon, _, _ = model(data)
        
        for i in range(n_samples):
            axes[row, i].imshow(recon[i].cpu().squeeze(), cmap='gray')
            axes[row, i].axis('off')
        axes[row, 0].set_ylabel(f'β={beta}', fontsize=10)
    
    plt.suptitle('不同 β 值的 VAE 重建比較')
    plt.tight_layout()
    plt.show()

compare_beta_reconstructions(beta_models, test_loader)

---
## 練習題

### 練習 1: 條件 VAE (CVAE)

**目標**: 實現條件 VAE，可以指定要生成的數字類別

**提示**: 將類別標籤編碼後與輸入/潛在變量連接

In [ ]:
# 練習 1: 條件 VAE
# Hint: 使用 one-hot 編碼將類別資訊加入編碼器和解碼器

class ConditionalVAE(nn.Module):
    """
    條件 VAE：可以指定生成特定類別
    
    修改：
    1. 編碼器輸入：x + 類別 one-hot
    2. 解碼器輸入：z + 類別 one-hot
    """
    
    def __init__(self, latent_dim: int = 32, num_classes: int = 10):
        super().__init__()
        
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        
        # 編碼器（輸入增加 num_classes 維度）
        self.encoder = nn.Sequential(
            nn.Linear(784 + num_classes, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
        )
        
        self.fc_mu = nn.Linear(256, latent_dim)
        self.fc_logvar = nn.Linear(256, latent_dim)
        
        # 解碼器（輸入增加 num_classes 維度）
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Sigmoid(),
        )
    
    def encode(self, x: torch.Tensor, c: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        編碼：x + 條件 -> (mu, log_var)
        
        Args:
            x: 輸入圖像 [B, 1, 28, 28]
            c: 類別標籤 [B] (整數)
        """
        # 展平圖像
        x_flat = x.view(x.size(0), -1)  # [B, 784]
        
        # One-hot 編碼類別
        c_onehot = F.one_hot(c, num_classes=self.num_classes).float()  # [B, num_classes]
        
        # 連接
        x_cond = torch.cat([x_flat, c_onehot], dim=1)  # [B, 784 + num_classes]
        
        h = self.encoder(x_cond)
        mu = self.fc_mu(h)
        log_var = self.fc_logvar(h)
        
        return mu, log_var
    
    def reparameterize(self, mu: torch.Tensor, log_var: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + std * eps
    
    def decode(self, z: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
        """
        解碼：z + 條件 -> x
        """
        c_onehot = F.one_hot(c, num_classes=self.num_classes).float()
        z_cond = torch.cat([z, c_onehot], dim=1)
        return self.decoder(z_cond)
    
    def forward(self, x: torch.Tensor, c: torch.Tensor):
        mu, log_var = self.encode(x, c)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z, c)
        return x_recon, mu, log_var

def train_cvae(model, train_loader, epochs=15, lr=1e-3):
    """訓練條件 VAE"""
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for data, labels in train_loader:
            data = data.to(device)
            labels = labels.to(device)
            
            recon, mu, log_var = model(data, labels)
            loss, _, _ = vae_loss(recon.view(-1, 1, 28, 28), data, mu, log_var)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader.dataset):.4f}")
    
    return model

# 訓練 CVAE
cvae = ConditionalVAE(latent_dim=32, num_classes=10).to(device)
cvae = train_cvae(cvae, train_loader, epochs=15)

In [ ]:
def generate_conditional_samples(model, n_per_class=5):
    """
    使用 CVAE 為每個類別生成樣本
    """
    model.eval()
    
    fig, axes = plt.subplots(10, n_per_class, figsize=(n_per_class * 1.5, 15))
    
    with torch.no_grad():
        for digit in range(10):
            # 為這個數字生成 n_per_class 個樣本
            z = torch.randn(n_per_class, model.latent_dim).to(device)
            c = torch.full((n_per_class,), digit, dtype=torch.long).to(device)
            
            samples = model.decode(z, c)
            
            for i in range(n_per_class):
                axes[digit, i].imshow(samples[i].cpu().view(28, 28), cmap='gray')
                axes[digit, i].axis('off')
            
            axes[digit, 0].set_ylabel(str(digit), fontsize=14, rotation=0, labelpad=15)
    
    plt.suptitle('條件 VAE 生成結果（每行一個數字）', fontsize=14)
    plt.tight_layout()
    plt.show()

generate_conditional_samples(cvae, n_per_class=8)

### 練習 2: VAE 異常檢測

**目標**: 使用 VAE 的重建誤差來檢測異常樣本

**提示**: 異常樣本的重建誤差通常較高

In [ ]:
# 練習 2: VAE 異常檢測

class VAEAnomalyDetector:
    """
    使用 VAE 進行異常檢測
    
    原理：
    - VAE 在正常資料上訓練
    - 對於異常資料，重建誤差會較高
    - 設定閾值來判斷是否為異常
    """
    
    def __init__(self, vae_model, threshold_percentile: float = 95):
        self.model = vae_model
        self.threshold = None
        self.threshold_percentile = threshold_percentile
    
    def compute_reconstruction_error(self, x: torch.Tensor) -> torch.Tensor:
        """計算重建誤差"""
        self.model.eval()
        with torch.no_grad():
            x_recon, mu, log_var = self.model(x)
            # 使用 MSE 作為重建誤差
            error = F.mse_loss(x_recon, x.view(x.size(0), -1), reduction='none')
            error = error.mean(dim=1)  # 每個樣本的平均誤差
        return error
    
    def fit_threshold(self, train_loader):
        """在正常資料上計算閾值"""
        all_errors = []
        
        for data, _ in train_loader:
            data = data.to(device)
            errors = self.compute_reconstruction_error(data)
            all_errors.append(errors.cpu())
        
        all_errors = torch.cat(all_errors)
        self.threshold = np.percentile(all_errors.numpy(), self.threshold_percentile)
        
        print(f"閾值設定為 {self.threshold_percentile}th 百分位數: {self.threshold:.6f}")
        return self.threshold
    
    def predict(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        預測是否為異常
        
        Returns:
            is_anomaly: 布林張量
            errors: 重建誤差
        """
        errors = self.compute_reconstruction_error(x)
        is_anomaly = errors > self.threshold
        return is_anomaly, errors

# 訓練一個只在某些數字上的 VAE
# 然後用其他數字作為「異常」

# 建立只包含 0-4 的訓練集
normal_digits = [0, 1, 2, 3, 4]
anomaly_digits = [5, 6, 7, 8, 9]

normal_indices = [i for i, (_, label) in enumerate(train_dataset) if label in normal_digits]
normal_subset = torch.utils.data.Subset(train_dataset, normal_indices)
normal_loader = DataLoader(normal_subset, batch_size=128, shuffle=True)

print(f"正常資料數量: {len(normal_subset)}")

# 訓練 VAE
anomaly_vae = VAE(latent_dim=16).to(device)
train_vae(anomaly_vae, normal_loader, epochs=10)

# 設定檢測器
detector = VAEAnomalyDetector(anomaly_vae, threshold_percentile=95)
detector.fit_threshold(normal_loader)

In [ ]:
# 測試異常檢測
def test_anomaly_detection(detector, test_loader, normal_digits, anomaly_digits):
    """
    測試異常檢測效果
    """
    normal_errors = []
    anomaly_errors = []
    
    for data, labels in test_loader:
        data = data.to(device)
        _, errors = detector.predict(data)
        errors = errors.cpu().numpy()
        labels = labels.numpy()
        
        for err, label in zip(errors, labels):
            if label in normal_digits:
                normal_errors.append(err)
            else:
                anomaly_errors.append(err)
    
    # 計算檢測率
    threshold = detector.threshold
    
    tp = sum(1 for e in anomaly_errors if e > threshold)  # 真正例：異常被檢測為異常
    fn = sum(1 for e in anomaly_errors if e <= threshold) # 假負例：異常被檢測為正常
    fp = sum(1 for e in normal_errors if e > threshold)   # 假正例：正常被檢測為異常
    tn = sum(1 for e in normal_errors if e <= threshold)  # 真負例：正常被檢測為正常
    
    print("\n異常檢測結果：")
    print(f"正常數字: {normal_digits}")
    print(f"異常數字: {anomaly_digits}")
    print(f"\n混淆矩陣:")
    print(f"           預測正常  預測異常")
    print(f"實際正常     {tn:5d}     {fp:5d}")
    print(f"實際異常     {fn:5d}     {tp:5d}")
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n精確率: {precision:.4f}")
    print(f"召回率: {recall:.4f}")
    print(f"F1 分數: {f1:.4f}")
    
    # 繪製誤差分佈
    plt.figure(figsize=(10, 4))
    plt.hist(normal_errors, bins=50, alpha=0.5, label=f'正常 ({normal_digits})', density=True)
    plt.hist(anomaly_errors, bins=50, alpha=0.5, label=f'異常 ({anomaly_digits})', density=True)
    plt.axvline(threshold, color='r', linestyle='--', label=f'閾值 ({threshold:.4f})')
    plt.xlabel('重建誤差')
    plt.ylabel('密度')
    plt.title('VAE 異常檢測 - 重建誤差分佈')
    plt.legend()
    plt.show()

test_anomaly_detection(detector, test_loader, normal_digits, anomaly_digits)

### 練習 3: 潛在空間算術

**目標**: 探索潛在空間中的語義運算（類似 word2vec 的 king - man + woman = queen）

In [ ]:
# 練習 3: 潛在空間算術

def get_class_mean_latent(model, data_loader, target_class):
    """
    計算特定類別在潛在空間中的平均位置
    """
    model.eval()
    latents = []
    
    with torch.no_grad():
        for data, labels in data_loader:
            mask = labels == target_class
            if mask.any():
                data = data[mask].to(device)
                mu, _ = model.encode(data)
                latents.append(mu.cpu())
    
    all_latents = torch.cat(latents, dim=0)
    mean_latent = all_latents.mean(dim=0)
    return mean_latent

def latent_arithmetic_demo(model, test_loader):
    """
    潛在空間算術示範
    
    例如：某個 3 - 平均的 3 + 平均的 8 = 看起來像 8 的結果
    """
    
    # 計算各類別的平均潛在向量
    class_means = {}
    for digit in range(10):
        class_means[digit] = get_class_mean_latent(model, test_loader, digit)
    
    # 取一個具體的樣本
    data, labels = next(iter(test_loader))
    
    # 找一個 3
    idx_3 = (labels == 3).nonzero()[0].item()
    sample_3 = data[idx_3:idx_3+1].to(device)
    
    model.eval()
    with torch.no_grad():
        z_sample_3, _ = model.encode(sample_3)
        
        # 算術：sample_3 - mean_3 + mean_8
        z_transformed = z_sample_3 - class_means[3].to(device) + class_means[8].to(device)
        
        # 解碼
        original = model.decode(z_sample_3)
        transformed = model.decode(z_transformed)
    
    # 可視化
    fig, axes = plt.subplots(1, 4, figsize=(10, 2.5))
    
    axes[0].imshow(sample_3.cpu().squeeze(), cmap='gray')
    axes[0].set_title('原始 (3)')
    axes[0].axis('off')
    
    axes[1].imshow(model.decode(class_means[3].unsqueeze(0).to(device)).cpu().squeeze().view(28, 28), cmap='gray')
    axes[1].set_title('平均 3')
    axes[1].axis('off')
    
    axes[2].imshow(model.decode(class_means[8].unsqueeze(0).to(device)).cpu().squeeze().view(28, 28), cmap='gray')
    axes[2].set_title('平均 8')
    axes[2].axis('off')
    
    axes[3].imshow(transformed.cpu().squeeze().view(28, 28), cmap='gray')
    axes[3].set_title('原始 - 平均3 + 平均8')
    axes[3].axis('off')
    
    plt.suptitle('潛在空間算術: 3 → 8 風格轉換')
    plt.tight_layout()
    plt.show()
    
    # 更多轉換示範
    print("\n更多風格轉換示範:")
    fig, axes = plt.subplots(4, 11, figsize=(14, 6))
    
    for row, source_digit in enumerate([1, 4, 7, 9]):
        # 找一個源數字樣本
        idx = (labels == source_digit).nonzero()[0].item()
        sample = data[idx:idx+1].to(device)
        
        with torch.no_grad():
            z_sample, _ = model.encode(sample)
        
        axes[row, 0].imshow(sample.cpu().squeeze(), cmap='gray')
        axes[row, 0].axis('off')
        axes[row, 0].set_title(f'原始 {source_digit}' if row == 0 else '')
        
        for col, target_digit in enumerate(range(10)):
            with torch.no_grad():
                z_transformed = z_sample - class_means[source_digit].to(device) + class_means[target_digit].to(device)
                transformed = model.decode(z_transformed)
            
            axes[row, col+1].imshow(transformed.cpu().squeeze().view(28, 28), cmap='gray')
            axes[row, col+1].axis('off')
            if row == 0:
                axes[row, col+1].set_title(f'→{target_digit}')
    
    plt.suptitle('潛在空間算術：行 = 源數字，列 = 目標風格')
    plt.tight_layout()
    plt.show()

latent_arithmetic_demo(vae_model, test_loader)

---
## 總結

### 本模組重點

1. **Autoencoder 基礎**
   - 編碼器壓縮、解碼器重建
   - 潛在空間捕捉資料本質特徵
   - 可用於降維、特徵學習

2. **VAE 關鍵改進**
   - 編碼器輸出分佈參數 (μ, σ)
   - 重參數化技巧使採樣可微分
   - KL 散度正則化潛在空間

3. **VAE 損失函數**
   ```python
   loss = reconstruction_loss + β * KL_divergence
   KL = -0.5 * sum(1 + log_var - mu^2 - exp(log_var))
   ```

4. **應用**
   - 生成新樣本
   - 潛在空間插值
   - 條件生成 (CVAE)
   - 異常檢測
   - 潛在空間算術

### 重要概念

```
普通 AE: x → z → x'
   VAE: x → (μ, σ) → z ~ N(μ, σ²) → x'
              ↓
         KL(N(μ,σ²) || N(0,1)) 正則化
```

### 下一步

在下一個模組中，我們將學習 **GAN 和 Diffusion 模型**，這是更強大的生成模型，能生成更高品質的圖像。

---

## Module 7 完整總結

### 🎯 核心概念

| 模型 | 潛在表示 | 損失函數 | 生成能力 |
|------|----------|----------|----------|
| **AE** | 確定性向量 z | 重建損失 | 差（空洞） |
| **VAE** | 分佈 N(μ, σ²) | 重建 + KL | 好（連續） |
| **β-VAE** | 分佈（更正則化）| 重建 + β·KL | 更解糾纏 |
| **CVAE** | 條件分佈 | 重建 + KL | 可控生成 |

### 🔑 關鍵公式

```
VAE 損失:
  L = E[log p(x|z)] - KL(q(z|x) || p(z))
    = Recon Loss + KL Divergence

KL 散度（高斯分佈）:
  KL = -0.5 * Σ(1 + log(σ²) - μ² - σ²)

重參數化技巧:
  z = μ + σ * ε, where ε ~ N(0, 1)
```

### 💡 實務技巧

1. **潛在維度選擇**：
   - MNIST: 2-32 維
   - 複雜圖像: 64-512 維
   - 太小會丟失資訊，太大會過擬合

2. **β 值調整**：
   - β < 1: 重建優先（更清晰但潛在空間可能不規整）
   - β = 1: 標準 VAE
   - β > 1: 潛在空間更規整（解糾纏）但可能模糊

3. **訓練穩定性**：
   - 使用 KL annealing（逐漸增加 KL 權重）
   - 監控 KL 是否 collapse（變成 0）

4. **架構建議**：
   - 圖像用卷積 VAE
   - 使用 BatchNorm 穩定訓練
   - 解碼器最後一層用 Sigmoid（如果輸入在 [0,1]）

### 🚀 下一步

- Module 8：GAN 與 Diffusion（更強大的生成模型）

---

**恭喜完成 VAE 模組！** 🎉

VAE 是生成模型的重要基石，它的潛在空間概念也是理解 Stable Diffusion 等現代生成模型的關鍵。掌握了 VAE，你就為學習更先進的生成模型打下了堅實基礎。